In [8]:
import math

In [9]:
def calculate_LJ(r_ij):
    """
    
    The lennard jones interaction energy btwn 2 particles. 
    
    
    Computes pairwise Lennard Jones interaction energy based on separation distnace in reduced units. 
    
    Parameters
    ---------------
    r_ij : float
        The disstance btwn particles in reduced units.
        
        
    Returns
    ----------
    pairwise_energy : float
        The parirwise Lennard JOnes interactin in reduced units.
    
        
    """
    
    r6_term = math.pow(1/r_ij, 6)
    r12_term = math.pow(r6_term, 2)
    pairwise_energy = 4 * (r12_term - r6_term)
    return pairwise_energy

In [10]:
def calculate_distance(coord1, coord2, box_length=None):
    """
    Calculate the distance between two points. When `box_length` is set, the minimum image convention is used to calculate the distance between the points.

    Parameters
    ----------
    coord1, coord2 : list
        The coordinates of the points, [x, y, z]
    
    box_length : float, optional
        The box length. This function assumes box is a cube.

    Returns
    -------
    distance : float
        The distance between the two points.
    """
    
    distance = 0
    
    for i in range(3):
        dim_dist = (coord1[i] - coord2[i])
        if box_length:
            dim_dist = (dim_dist - box_length * round(dim_dist/ box_length))**2
        distance += dim_dist
        
        
    distance = math.sqrt(distance)
    
    return distance


In [11]:
def calculate_total_energy(coordinates, cutoff, box_length=None):
    total_energy = 0
    num_atoms = len(coordinates)

    for i in range(num_atoms):
        for j in range(i+1, num_atoms):
            dist_ij = calculate_distance(coordinates[i], coordinates[j], box_length)
            if dist_ij < cutoff:
                pairwise_energy = calculate_LJ(dist_ij)
            
                total_energy += pairwise_energy
    
    return total_energy

In [12]:
# Provided function

def read_xyz(filepath):
    """
    Reads coordinates from an xyz file.
    
    Parameters
    ----------
    filepath : str
       The path to the xyz file to be processed.
       
    Returns
    -------
    atomic_coordinates : list
        A two dimensional list containing atomic coordinates
    """
    
    with open(filepath) as f:
        box_length = float(f.readline().split()[0])
        num_atoms = float(f.readline())
        coordinates = f.readlines()
    
    atomic_coordinates = []
    
    for atom in coordinates:
        split_atoms = atom.split()
        
        float_coords = []
        
        # We split this way to get rid of the atom label.
        for coord in split_atoms[1:]:
            float_coords.append(float(coord))
            
        atomic_coordinates.append(float_coords)
        
    
    return atomic_coordinates, box_length

In [13]:
## Read in first sample configuration file here

import os

config1_file = os.path.join("../../lj_sample_configurations", "lj_sample_config_periodic1.txt")

sample_coords, box_length = read_xyz(config1_file)


In [14]:
## Assertions here
total_energy = calculate_total_energy(sample_coords,3,box_length)

print(total_energy)
assert math.isclose(total_energy,-4351.5,rel_tol=1e-03)